In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from model import Iwata_simple
from data import IWATA_Classification_Sampler
import numpy as np
import time

from utils import StandardScaler
import pandas as pd
import psycopg2 as pg
import tqdm

import os
import sktime
from sktime.datasets import load_from_tsfile_to_dataframe
from utils_iwata.preprocess import preprocess_iwata
import random

C:\Users\Stonker69\Documents\GitHub\BiksStonks\Forcasting_Models\env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Preprocess 

In [2]:
# preprocess_iwata(data_list_path='data_list.csv', 
#                  out_ds_path='preprocessed_iwata_ds', 
#                  original_ds_path='Univariate_ts')

## Construct Model

In [3]:
bidirectional = True
seq_len=13
pred_len=1 # must be one fore Iwata_simple
enc_in = 1
hidden_size = 64
c_out = 1
s_n_layers = 2
batch_size = 32 # = support size
direcs = 2 if bidirectional else 1
model = Iwata_simple(enc_in, hidden_size, c_out, s_n_layers)
support_set = torch.rand(batch_size, seq_len, enc_in)
query_set = torch.rand(1, seq_len, enc_in)
output = model(support_set, query_set)
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
seed = 0

In [4]:
print(model)

Iwata_simple(
  (support_encoder): LSTM(1, 32, num_layers=2, bidirectional=True)
  (query_encoder): LSTM(1, 64)
  (attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
  )
  (g): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
)


## Train

In [12]:
class_ds = IWATA_Classification_Sampler(root_path='preprocessed_iwata_ds', 
                                        seq_len=seq_len, pred_len=pred_len, num_samples=1000,
                                        flag='TRAIN', seed=seed)
data_loader = DataLoader(
            class_ds,
            batch_size=1, # only works with one as they are sampled already from Q_N, S_N
            shuffle=False,
            drop_last=True)

55 10 24


In [5]:
epochs = 10

In [20]:
model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
mse = nn.MSELoss()

In [7]:
train_steps = len(data_loader)
time_now = time.time()

for epoch in range(epochs):
    train_loss = []
    epoch_time = time.time()
    iter_count = 0
    for s_seq_x, q_seq_x, q_seq_y in data_loader:
        s_seq_x = s_seq_x.float().squeeze(0).to(device) # support set
        q_seq_x = q_seq_x.float().squeeze(0).to(device) # query set 
        q_seq_y = q_seq_y.float() # query set label 

        optimizer.zero_grad()
        output = model(s_seq_x, q_seq_x)
        loss = mse(output, q_seq_y)
        train_loss.append(loss.item())
        loss.backward()
        optimizer.step()
        iter_count += 1
        if (iter_count) % 100 == 0: 
            print("\titers: {0}, epoch: {1} | loss: {2:.7f}".format(iter_count, epoch + 1, loss.item()))
            speed = (time.time()-time_now)/iter_count
            left_time = speed*((epochs - epoch)*train_steps - iter_count)
            print('\tspeed: {:.4f}s/iter; left time: {:.4f}s'.format(speed, left_time))
            iter_count = 0
            time_now = time.time()

train_loss = np.average(train_loss)
print('Epoch {}/{} \t Time: {:.2f}s \t Loss: {:.4f}'.format(epoch+1, epochs, time.time() - epoch_time, train_loss))

	iters: 100, epoch: 1 | loss: 0.8599555
	speed: 0.0186s/iter; left time: 183.8604s
	iters: 100, epoch: 1 | loss: 0.0964473
	speed: 0.0182s/iter; left time: 180.2401s
	iters: 100, epoch: 1 | loss: 2.0093324
	speed: 0.0183s/iter; left time: 181.1427s
	iters: 100, epoch: 1 | loss: 0.0099179
	speed: 0.0181s/iter; left time: 179.5575s
	iters: 100, epoch: 1 | loss: 0.0000070
	speed: 0.0190s/iter; left time: 188.5607s
	iters: 100, epoch: 1 | loss: 0.0208042
	speed: 0.0179s/iter; left time: 177.4838s
	iters: 100, epoch: 1 | loss: 0.0575268
	speed: 0.0181s/iter; left time: 178.9595s
	iters: 100, epoch: 1 | loss: 0.0154225
	speed: 0.0182s/iter; left time: 179.9302s
	iters: 100, epoch: 1 | loss: 0.0306734
	speed: 0.0180s/iter; left time: 177.9566s
	iters: 100, epoch: 1 | loss: 0.5926484
	speed: 0.0180s/iter; left time: 178.2969s
	iters: 100, epoch: 2 | loss: 0.0192931
	speed: 0.0177s/iter; left time: 157.6714s
	iters: 100, epoch: 2 | loss: 0.0002157
	speed: 0.0187s/iter; left time: 166.1436s
	ite

## Save Model

In [13]:
# save torch model 
path = 'checkpoints'
if not os.path.exists(path):
    os.mkdir(path)
model_specific_tag = f'seed{seed},enc_in{enc_in},hidden_size{hidden_size},c_out{c_out},s_n_layers{s_n_layers}'
model_name = 'iwata_simple_benchmark_' + model_specific_tag + '.pt'
# torch.save(model.state_dict(), os.path.join(path, model_name))

## Load Model

In [14]:
# load saved weights 

model.load_state_dict(torch.load(os.path.join(path, model_name)))

<All keys matched successfully>

## Evaluate model

In [15]:
data_list = os.listdir('preprocessed_iwata_ds')

In [16]:
def count_test_timeseries(data_list):
    n = 0
    for data in data_list:
        X,_ = load_from_tsfile_to_dataframe(f'preprocessed_iwata_ds/{data}/{data}_TEST.ts')
        n += len(X)
    return n
total_rows = count_test_timeseries(class_ds.target_tasks) # this can take a couple of secs

In [17]:
samples_per_series = 100-seq_len-pred_len+1 
percent_to_use = .5
terminate_at = int(samples_per_series*percent_to_use)

percent_of_rows_to_use = .3

total_series = terminate_at * int(total_rows*percent_of_rows_to_use)
series_evaluated = 0

class_ds = IWATA_Classification_Sampler(root_path='preprocessed_iwata_ds', 
                                        seq_len=seq_len, pred_len=pred_len, num_samples=total_series,
                                        flag='TRAIN', seed=seed)

data_loader = DataLoader(
            class_ds,
            batch_size=1, # only works with one as they are sampled already from Q_N, S_N
            shuffle=False,
            drop_last=True)
d_iter = iter(data_loader)

55 10 24


In [18]:
print(f'Terminate at {terminate_at} - sample so many timeseries per row')
print(f'Total number of rows: {total_rows} - {int(total_rows*percent_of_rows_to_use)} rows will be used')
print(f'Total number of series: {total_series}')

Terminate at 43 - sample so many timeseries per row
Total number of rows: 23484 - 7045 rows will be used
Total number of series: 302935


In [21]:
model.eval()
metrics = dict()

for data in class_ds.target_tasks:
    metrics[data] = []
    print('Evaluating on {}'.format(data))
    X,_ = load_from_tsfile_to_dataframe(f'preprocessed_iwata_ds/{data}/{data}_TEST.ts')
    # sample subset of rows 
    n = len(X['dim_0'])
    rows_to_use = int(n*percent_of_rows_to_use)
    X = X.sample(rows_to_use)

    loss = 0
    samples_per_data = 0
    pbar = tqdm.tqdm(n, desc=f'Evaluating timeseries for {data}', total=len(X), unit='rows')
    for row in X['dim_0']:
        row = row.to_numpy()
        windows = np.lib.stride_tricks.sliding_window_view(row, window_shape=seq_len+pred_len)
        windows = random.sample(list(windows), min(terminate_at, len(windows))) # sample 
        # sample once per row
        s_seq_x, _, _ = next(d_iter) # advance the iterator
        for i in range(0, min(terminate_at, len(windows))):
            x, y = windows[i][:seq_len], windows[i][seq_len:]
            series_evaluated += 1
            # evaluate here 
            q_seq_x = torch.tensor(x[None,...,None]).float().to(device)
            q_seq_y = torch.tensor(y).float().to(device)
            s_seq_x = s_seq_x.float().squeeze(0).to(device)
            output = model(s_seq_x, q_seq_x)
            loss += mse(output, q_seq_y)
            samples_per_data += 1
        pbar.update(1)
    pbar.close()

    metrics[data] = [loss/samples_per_data, torch.sqrt(loss/samples_per_data)]

Evaluating on ArrowHead


Evaluating timeseries for ArrowHead: 100%|██████████| 52/52 [00:14<00:00,  3.57rows/s]


## Save benchmark

In [25]:
import csv

with open(f'iwata_few_shot_benchmark_{model_specific_tag}.csv', 'w', newline='') as csvfile:
    fieldnames = ['dataset name', 'MSE', 'RMSE']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for key, val in metrics.items():
        writer.writerow({'dataset name': key, 'MSE': val[0].item(), 'RMSE': val[1].item()})